In [43]:
import torch
import yaml

In [44]:
with open("./sample.yaml", "r") as file:
    config = yaml.safe_load(file)

In [45]:
config['model']

{'name': 'SimpleObjectDetectionModel',
 'type': 'CNN',
 'input_shape': [3, 224, 224],
 'num_classes': 20,
 'layers': [{'type': 'Conv2D',
   'filters': 32,
   'kernel_size': [3, 3],
   'strides': [1, 1],
   'padding': 'same',
   'activation': 'ReLU'},
  {'type': 'MaxPooling2D', 'pool_size': [2, 2], 'strides': [2, 2]},
  {'type': 'Conv2D',
   'filters': 64,
   'kernel_size': [3, 3],
   'strides': [1, 1],
   'padding': 'same',
   'activation': 'ReLU'},
  {'type': 'MaxPooling2D', 'pool_size': [2, 2], 'strides': [2, 2]},
  {'type': 'Conv2D',
   'filters': 128,
   'kernel_size': [3, 3],
   'strides': [1, 1],
   'padding': 'same',
   'activation': 'ReLU'},
  {'type': 'MaxPooling2D', 'pool_size': [2, 2], 'strides': [2, 2]},
  {'type': 'Flatten'},
  {'type': 'Dense', 'units': 512, 'activation': 'ReLU'},
  {'type': 'Dropout', 'rate': 0.5},
  {'type': 'Dense', 'units': 'num_classes', 'activation': 'Softmax'}]}

In [46]:
config["model"]["layers"][0]["type"]

'Conv2D'

In [63]:
class sampleModel(torch.nn.Module):
    def __init__(self):
        super(sampleModel, self).__init__()
        layers = []
        input_shape = config["model"]["input_shape"]
        for layer_cfg in config["model"]["layers"]:
            if layer_cfg["type"] == 'Conv2D':
                layers.append(torch.nn.Conv2d(
                    in_channels = input_shape[0],
                    out_channels = layer_cfg["filters"],
                    kernel_size = tuple(layer_cfg["kernel_size"]),
                    stride= tuple(layer_cfg["strides"]),
                    padding= layer_cfg["padding"])
                    )
                layers.append(getattr(torch.nn, layer_cfg["activation"])())
                input_shape[0] = layer_cfg["filters"]
            elif layer_cfg["type"] == 'MaxPooling2D':
                layers.append(torch.nn.MaxPool2d(
                    kernel_size=tuple(layer_cfg["pool_size"]),
                    stride=tuple(layer_cfg["strides"])
                ))
            elif layer_cfg["type"] == "Flatten":
                layers.append(torch.nn.Flatten())
            elif layer_cfg["type"] == "Dense":
                if layer_cfg["units"] == 'num_classes':
                    layer_cfg["units"] = config["model"]["num_classes"]
                layers.append(torch.nn.Linear(input_shape[0], layer_cfg["units"]))
                # print(layer_cfg["activation"])
                layers.append(getattr(torch.nn, layer_cfg["activation"])())
                input_shape[0] = layer_cfg["units"]
            elif layer_cfg["type"] == "Dropout":
                layers.append(torch.nn.Dropout(p=layer_cfg["rate"]))
        self.model = torch.nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [64]:
samplemodel = sampleModel()

In [62]:
for i , layer in enumerate(samplemodel.model.children()):
    print(i ,layer)

0 Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
1 ReLU()
2 MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
3 Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
4 ReLU()
5 MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
6 Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
7 ReLU()
8 MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
9 Flatten(start_dim=1, end_dim=-1)
10 Linear(in_features=128, out_features=512, bias=True)
11 ReLU()
12 Dropout(p=0.5, inplace=False)
13 Linear(in_features=512, out_features=20, bias=True)
14 Softmax(dim=None)
